In [1]:
import pandas as pd

df = pd.read_csv('../data/data-binary.csv')
df.dtypes

a    int64
b    int64
c    int64
d    int64
e    int64
dtype: object

In [2]:
df.head()

,a,b,c,d,e
0,1,0,0,0,0
1,1,0,0,0,0
2,1,0,0,1,1
3,0,0,0,0,1
4,0,0,0,0,0


In [3]:
from pypair.association import binary_binary
import numpy as np

gkl_for = lambda a, b: binary_binary(a, b, measure='gk_lambda')
gkl_rev = lambda a, b: binary_binary(a, b, measure='gk_lambda_reversed')

gkl_df = pd.DataFrame(
    np.triu(df.corr(method=gkl_for), k=1) + np.tril(df.corr(method=gkl_rev), k=-1),
    columns=df.columns,
    index=df.columns
)

gkl_df

,a,b,c,d,e
a,0.0,0.0,0.000000,0.000000,0.000000
b,0.0,0.0,0.357166,0.000000,0.000000
c,0.0,0.0,0.000000,0.000000,0.000000
d,0.0,0.0,0.000000,0.000000,0.622583
e,0.0,0.0,0.000000,0.523902,0.000000


In [4]:
from itertools import chain

def get_larger(i, j):
    ij = gkl_df.iloc[i, j]
    jk = gkl_df.iloc[j, i]
    
    if ij == jk:
        return None
    elif ij > jk:
        return i, j
    else:
        return j, i
    
def to_name(i, j):
    return gkl_df.columns[i], gkl_df.columns[j]

    
indices = [(i, list(r.to_numpy().nonzero()[0])) for i, (_, r) in enumerate(gkl_df.iterrows())]
indices = map(lambda tup: [(tup[0], i) for i in tup[1]], indices)
indices = filter(lambda arr: len(arr) > 0, indices)
indices = chain(*indices)
indices = map(lambda tup: (tup[0], tup[1]) if tup[0] < tup[1] else (tup[1], tup[0]), indices)
indices = sorted(list(set(indices)))
indices = map(lambda tup: get_larger(tup[0], tup[1]), indices)
indices = filter(lambda tup: tup is not None, indices)
indices = map(lambda tup: to_name(tup[0], tup[1]), indices)

list(indices)

[('b', 'c'), ('d', 'e')]